In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = (
    SparkSession.builder.appName("ADS project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/18 19:10:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
import pandas as pd
import numpy as np

raw_data = "../data/ABS/"
curated_data = "../data/curated/"

tbl_income = pd.read_excel(raw_data+"annual income by SA2 regions.xlsx")
tbl_population = pd.read_excel(raw_data+"population estimates by SA2 regions.xlsx")


<h2>1. Clean external data</h2>

<h3>1.1 Clean "annual income"</h3>

In [3]:
tbl_income.drop(np.arange(0,8), inplace=True)
tbl_income.drop(np.arange(2303,2306), inplace=True)

tbl_income.drop(tbl_income.columns[[2,3,4,5,7,8,9,10,12,13,14,15,17,18,19,20,22,23,24,25,27]], axis=1, inplace=True)
tbl_income = tbl_income.reset_index(drop=True)
tbl_income.columns = ['sa2', 'sa2_name', 'total_earners', 'median_age', 'income_sum', 'income_median', 'income_mean']

In [4]:
tbl_income = tbl_income[tbl_income["total_earners"] != "np"]
tbl_income["total_earners"] = pd.to_numeric(tbl_income["total_earners"])
tbl_income["median_age"] = pd.to_numeric(tbl_income["median_age"])
tbl_income["income_sum"] = pd.to_numeric(tbl_income["income_sum"])
tbl_income["income_median"] = pd.to_numeric(tbl_income["income_median"])
tbl_income["income_mean"] = pd.to_numeric(tbl_income["income_mean"])

In [5]:
tbl_income.to_csv(curated_data+"clean_annual.csv")

<h3>1.2 Clean "population"</h3>

In [6]:
tbl_population.drop(tbl_population.columns[0:8], axis=1, inplace=True)
tbl_population.drop(tbl_population.columns[1:22], axis=1, inplace=True)
tbl_population.drop(tbl_population.columns[2:6], axis=1, inplace=True)

In [7]:
tbl_population.columns = ["sa2", "2021_population", "km2", "persons/km2"]
tbl_population = tbl_population.dropna(axis=0, how='any', thresh=None, subset="sa2", inplace=False)
tbl_population['sa2'] = tbl_population['sa2'].astype(float).astype(int)
tbl_population

,sa2,2021_population,km2,persons/km2
0,101021007,4330.0,3418.4,1.3
1,101021008,8546.0,7.0,1223.9
2,101021009,11370.0,4.8,2387.7
3,101021010,5093.0,13.0,391.7
4,101021012,12743.0,13.7,931.9
...,...,...,...,...
2449,801111141,67.0,1202.8,0.1
2450,901011001,1716.0,136.1,12.6
2451,901021002,602.0,13.7,43.9
2452,901031003,310.0,67.2,4.6


In [8]:
tbl_population.to_csv(curated_data+"clean_population.csv")

<h3>1.3 Clean "postcode_sa2"</h3>

In [9]:
postcode_2011_SA2_2011 = pd.ExcelFile(raw_data+"1270055006_CG_POSTCODE_2011_SA2_2011.xls")
postcode_sa2 = pd.read_excel(postcode_2011_SA2_2011, "Table 3")

In [10]:
postcode_sa2.drop(np.arange(0,6), inplace=True)
postcode_sa2.drop(np.arange(5994,5997), inplace=True)

postcode_sa2.drop(postcode_sa2.columns[3:6], axis=1, inplace=True)
postcode_sa2.drop(postcode_sa2.columns[0], axis=1, inplace=True)
postcode_sa2 = postcode_sa2.reset_index(drop=True)

postcode_sa2.columns = ['postcode', 'sa2']
postcode_sa2['sa2'] = pd.to_numeric(postcode_sa2['sa2'])

In [11]:
postcode_sa2.to_csv(curated_data+"postcode_sa2.csv")

<h2>2. Join external data</h2>

<h3>2.1 Join "annual income" and "population"</h3>

In [12]:
merge_income_population = tbl_income.merge(tbl_population, on='sa2', how='inner')

In [13]:
merge_income_population

,sa2,sa2_name,total_earners,median_age,income_sum,income_median,income_mean,2021_population,km2,persons/km2
0,101021007,Braidwood,2361,51,120763285,41593,51149,4330.0,3418.4,1.3
1,101021008,Karabar,5100,42,338308979,61777,66335,8546.0,7.0,1223.9
2,101021009,Queanbeyan,6697,39,441160946,60119,65874,11370.0,4.8,2387.7
3,101021010,Queanbeyan - East,3393,40,237035656,63051,69860,5093.0,13.0,391.7
4,101021012,Queanbeyan West - Jerrabomberra,8476,44,725602722,73851,85607,12743.0,13.7,931.9
...,...,...,...,...,...,...,...,...,...,...
2099,801101135,Coombs,2270,35,179834174,74179,79222,4834.0,2.3,2085.6
2100,801101136,Denman Prospect,416,38,41056005,90337,98692,2719.0,4.7,573.2
2101,801101139,Wright,2121,35,182421537,79150,86007,3806.0,1.3,2993.1
2102,801111140,ACT - South West,355,40,26069449,64227,73435,554.0,416.8,1.3


<h3>2.2 Join "merge_income_population" and "postcode_sa2"</h3>

In [14]:
merge_income_population_postcode = merge_income_population.merge(postcode_sa2, on='sa2', how='left')
merge_income_population_postcode = merge_income_population_postcode.dropna(axis=0, how='any', thresh=None, subset="postcode", inplace=False)

In [15]:
agg_income_population_postcode = merge_income_population_postcode.groupby("postcode").agg(
    {
    "total_earners": "sum",
    "median_age": "mean",
    "income_sum": "sum",
    "2021_population": "sum",
    "km2": "sum"
    }
).reset_index()

In [16]:
agg_income_population_postcode

,postcode,total_earners,median_age,income_sum,2021_population,km2
0,0800,5632,33.000000,420609031,7679.0,3.2
1,0810,21932,39.583333,1574969237,36216.0,24.3
2,0812,11443,42.000000,801236575,19888.0,12.0
3,0820,15219,39.555556,1235996605,22877.0,67.8
4,0822,21869,40.142857,1425664118,61406.0,248437.3
...,...,...,...,...,...,...
2416,7466,1985,45.000000,115705864,4373.0,3931.6
2417,7467,1985,45.000000,115705864,4373.0,3931.6
2418,7468,1985,45.000000,115705864,4373.0,3931.6
2419,7469,1985,45.000000,115705864,4373.0,3931.6


<h3>2.3 Join "agg_income_population_postcode" and "mechant_consumer_info" (internal data)</h3>

In [17]:
merchant_consumer_info = spark.read.parquet(curated_data+"mechant_consumer_info")

In [18]:
# merchant_consumer_info["consumer_postcode"] = merchant_consumer_info["consumer_postcode"].str.pad(width=4, fillchar="0", side="left")
merchant_consumer_info = merchant_consumer_info.withColumn("consumer_postcode", lpad(col("consumer_postcode"), 4, "0"))

In [19]:
agg_income_population_postcode = spark.createDataFrame(agg_income_population_postcode) 

In [21]:
agg_income_population_postcode.createOrReplaceTempView('agg_income_population_postcode')
merchant_consumer_info.createOrReplaceTempView('merchant_consumer_info')

final_table = spark.sql("""
SELECT 
    *
FROM 
    agg_income_population_postcode
RIGHT JOIN
    merchant_consumer_info
ON 
    agg_income_population_postcode.postcode = merchant_consumer_info.consumer_postcode
""")

In [32]:
(final_table.where(col("income_sum").isNull())).select(countDistinct("consumer_postcode"))
# print(f"Mismatched (null) transactions dropped: {merchant_consumer_info.shape[0] - final_table.shape[0]}")

count(DISTINCT consumer_postcode)
746


In [33]:
final_table

postcode,total_earners,median_age,income_sum,2021_population,km2,merchant_name,products,revenue_level,take_rate,dollar_value,order_datetime,consumer,consumer_address,consumer_state,consumer_postcode,consumer_gender,__index_level_0__
2323,17295,42.666666666666664,1016410771,35881.0,1933.0,Magna Sed Institute,"[cable, satellite...",e,0.31,9.737964095722326,2021-05-10,Brian Barnett,989 Robinson Street,NSW,2323,Male,3
2323,17295,42.666666666666664,1016410771,35881.0,1933.0,Et Nunc Consulting,"[books, periodica...",e,0.16,770.1754014299855,2021-11-19,Brian Barnett,989 Robinson Street,NSW,2323,Male,4
2396,3642,49.0,137575970,7946.0,10474.1,Nunc Est LLP,"[watch, clock, an...",e,0.22,186.91415986083496,2021-05-27,Roberto Robbins,3146 Eric Turnpik...,NSW,2396,Male,5
2396,3642,49.0,137575970,7946.0,10474.1,Felis Limited,"[furniture, home,...",e,0.18,74.68707218671928,2021-08-11,Roberto Robbins,3146 Eric Turnpik...,NSW,2396,Male,6
2396,3642,49.0,137575970,7946.0,10474.1,Metus Sit Amet In...,"[cable, satellite...",e,0.38,20.49850981713674,2021-07-04,Roberto Robbins,3146 Eric Turnpik...,NSW,2396,Male,7
2396,3642,49.0,137575970,7946.0,10474.1,Elit Consulting,"[antique, shops, ...",e,0.47,316.7257267436573,2021-03-05,Roberto Robbins,3146 Eric Turnpik...,NSW,2396,Male,8
2396,3642,49.0,137575970,7946.0,10474.1,Nullam Vitae Diam PC,"[opticians, optic...",e,0.22,72.77954022414646,2021-09-13,Roberto Robbins,3146 Eric Turnpik...,NSW,2396,Male,9
2396,3642,49.0,137575970,7946.0,10474.1,Metus Sit Amet In...,"[cable, satellite...",e,0.38,57.27484630595788,2022-08-20,Roberto Robbins,3146 Eric Turnpik...,NSW,2396,Male,10
null,null,null,null,null,null,Et Nunc Consulting,"[books, periodica...",e,0.16,147.74061765538696,2021-12-09,Allison Stevens,60866 Miller Avenue,NSW,1193,Female,11
null,null,null,null,null,null,Dis Parturient Co...,"[gift, card, nove...",e,0.24,140.36239513919142,2021-11-08,Allison Stevens,60866 Miller Avenue,NSW,1193,Female,12


In [26]:
final_table.write.parquet(curated_data+"transact_abs.parquet")